In [ ]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord


Entrez.email = "something@something.com"

handle = Entrez.esearch(db="nucleotide", term="mitochondrion, complete genome", RetMax=10000)
record = Entrez.read(handle)
# print(record["IdList"])
# print(record["Count"])
gi_list = record["IdList"]
handle = Entrez.efetch(db="nucleotide", id=gi_list, rettype="gb", retmode="text")
text = handle.read()
# print(text)

records = SeqIO.parse(handle, "gb")
handle = Entrez.efetch(db="nucleotide", id=gi_list, rettype="gb", retmode="text")
records = SeqIO.parse(handle, "gb")
#with open("records.fasta", "w") as f:
#   SeqIO.write(records, f, "fasta")
        

In [58]:
import sys
import re


len_fasta_out_file = 60
def module_fasta_writer(out_file, line):  # writes in fasta format by 60 nums in each line
    start_point = 0
    end_point = len_fasta_out_file
    if len(line) > len_fasta_out_file:
        len_line = len(line)
        while len_line > 0:
            j = line[start_point:end_point]
            len_line = len_line - len_fasta_out_file
            out_file.write(j + "\n")
            start_point = start_point + len_fasta_out_file
            end_point = end_point + len_fasta_out_file
    else:
        out_file.write(line + "\n")
        
        
def GC_module(sequence):
    nuc_collector = {}
    for i in range(len(sequence)):
        nuc = sequence[i]
        if nuc in nuc_collector.keys():
            nuc_collector[nuc] = int(nuc_collector[nuc]) + 1
        else:
            nuc_collector[nuc] = 1
    GC = nuc_collector["G"]+nuc_collector["C"]/len(sequence)
    return round(GC,3)

stop = 0
with open("records.fasta", "r") as f:
    seqs={}
    for line in f:
        line = line.rstrip()
        #print(line)
        if len(line) > 0:
            if line[0] == ">":
                name = line.strip()
                seqs[name] = ''
                stop += 1
                if stop == 100000:
                    break
            else:
                seqs[name] = seqs[name] + line
        # print(seqs)
                
names = list(seqs.keys())

In [64]:
# print(" ".join(names[0].split()[-3:]))
# print(len(names))
new_list = [x for x in names if re.fullmatch(r"mitochondrion, complete genome", " ".join(x.split()[-3:]))]
# print(len(new_list))
name_list = [" ".join(x.split()[1:3]) for x in new_list]
# print(name_list)
# new_list.append(new_list1)
# new_new_list = [x for x in new_list if "EC=1.1" in x]

from itertools import groupby
sorted_list = [el for el, _ in groupby(name_list)]
# print(sorted_list)

dic = {}
for i in sorted_list:
    # print(i)
    for j in names:
        if i == " ".join(j.split()[1:3]):
            if i in dic.keys():
                dic[i] = dic[i] + 1
            if i not in dic.keys():
                dic[i] = 1
# print(dic)
# sorted(dic.items(), key=lambda x: -x[1])

callul = 0
new_dic = {}
for i in dic.keys():
    if dic[i] >= 4: 
        callul += dic[i]
        new_dic[i] = dic[i]
        # print(dic[i])
        
# print(new_dic)
print("Найдено организмов:", len(new_dic.keys()))
print("Всего последовательностей:", callul)

Найдено организмов: 715
Всего последовательностей: 411440


In [62]:
import os
directory =" files"
os.mkdir(directory)
os.chdir(directory) 

In [63]:
for i in new_dic.keys():
    with open(str(i) + ".fasta", 'w') as f:
        for j in seqs.keys():
            if " ".join(j.split()[1:3]) == i:
                f.write(j + "\n")
                module_fasta_writer(f, seqs[j])